## Pipeline for microendoscopic data processing in CaImAn using the CNMF-E algorithm
<p><img src="../../docs/img/quickintro.png" /></p>
This demo presents a complete pipeline for processing microendoscopic data using CaImAn. It includes:
- Motion Correction using the NoRMCorre algorithm
- Source extraction using the CNMF-E algorithm
- Deconvolution using the OASIS algorithm

Some basic visualization is also included. The demo illustrates how to `params`, `MoctionCorrection` and `cnmf` object for processing 1p microendoscopic data. For processing two-photon data consult the related `demo_pipeline.ipynb` demo. For more information see the companion CaImAn paper.

In [1]:
#!/usr/bin/env python

try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import logging
import matplotlib.pyplot as plt
import numpy as np

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
#                     filename="C:/Users/BlairLab-01/CaImAn/test/test_caiman.log",
                    level=logging.DEBUG)

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2

try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl
import holoviews as hv
#bpl.output_notebook()
#hv.notebook_extension('bokeh')


### GJB additions
logging.getLogger('matplotlib.font_manager').disabled = True
import os
# import scipy.io
# from LFOV_functions import convert_behav
# from LFOV_functions import crop_and_convert_miniscope as covertAvis
# from LFOV_functions import get_all_CropROIs as get_crop
# from LFOV_functions import extract_position
import glob
from tifffile import TiffWriter, imread, imwrite
import scipy

C:\Users\gjb326\AppData\Local\Temp\ipykernel_9564\891152385.py:4: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic(u'load_ext autoreload')
C:\Users\gjb326\AppData\Local\Temp\ipykernel_9564\891152385.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic(u'autoreload 2')
C:\Users\gjb326\AppData\Local\Temp\ipykernel_9564\891152385.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic(u'matplotlib qt')
       31665 [__init__.py:            <module>():47] [9564] Creating converter from 7 to 5
       31665 [__init__.py:            <module>():47] [9564] Creating converter from 5 to 7
       31666 [__init__.py:            <module>():47] [9564] Creating converter from 7 to 5

In [2]:
from miniscope_tiffs import crop_and_convert_miniscope, get_all_CropROIs, getVideosInFolder, GetMiniscopeDirs

# animals = ['Hipp16942'] # ['Hipp16941', 'Hipp16942', 'Hipp16943']
# topdir = 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/'

animals = ['TestMouse1']
topdir = 'C:/Users/gjb326/Desktop/RecordingData/AlejandroGrau/'
# miniscope_tiff_name = 'msCam.tiff' 
miniscope_tiff_name = 'msCam.tiff' # this will allow to select directories that do or do not contain this file
behavcam_tiff_name = 'behavCam.tiff' 
TiffOnly = None

allMiniFolders, allBehavFolders, parentMiniFolders, parentBehavFolders = \
    GetMiniscopeDirs(topdir, animals, TiffDirsOnly=TiffOnly, miniscope_tiff_name=miniscope_tiff_name, behavcam_tiff_name=behavcam_tiff_name)

allMiniFolders

['C:/Users/gjb326/Desktop/RecordingData/AlejandroGrau/TestMouse1/2022_05_07/17_09_41/MiniLFOV']

### Select file(s) to be processed
The `download_demo` function will download the specific file for you and return the complete path to the file which will be stored in your `caiman_data` directory. If you adapt this demo for your data make sure to pass the complete path to your file(s). Remember to pass the `fnames` variable as a list. Note that the memory requirement of the CNMF-E algorithm are much higher compared to the standard CNMF algorithm. Test the limits of your system before trying to process very large amounts of data.

### Setup a cluster
To enable parallel processing a (local) cluster needs to be set up. This is done with a cell below. The variable `backend` determines the type of cluster used. The default value `'local'` uses the multiprocessing package. The `ipyparallel` option is also available. More information on these choices can be found [here](https://github.com/flatironinstitute/CaImAn/blob/master/CLUSTER.md). The resulting variable `dview` expresses the cluster option. If you use `dview=dview` in the downstream analysis then parallel processing will be used. If you use `dview=None` then no parallel processing will be employed.

In [3]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=12, single_thread=False)

### Setup some parameters
We first set some parameters related to the data and motion correction and create a `params` object. We'll modify this object with additional settings later on. You can also set all the parameters at once as demonstrated in the `demo_pipeline.ipynb` notebook.

In [5]:
frate = 11                       # movie frame rate
decay_time = 2                 # length of a typical transient in seconds, 0.4 default
#1 prev
# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = True         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (2,2) # (3,3)       # size of high pass spatial filtering, used in 1p data FOR HIPP7
max_shifts = (55, 55)      # maximum allowed rigid shift #default (5,5)
strides = (128, 128)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (32, 32)      # overlap between pathes (size of patch strides+overlaps)
max_deviation_rigid = 3#3  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries
# dview = None


# parameters for source extraction and deconvolution
p = 1#2?               # order of the autoregressive system ########## default 1
K = None            # upper bound on number of components per patch, in general None
gSig = (4,4)       # gaussian width of a 2D gaussian kernel, which approximates a neuron default=(3,3) # (3,3) for hipp6-9
                    # expected half size of neurons
    # (4,4) looks very good for Hipp16942
gSiz = (13, 13)     # average diameter of a neuron, in general 4*gSig+1 default=(13,13)
Ain = None          # possibility to seed with predetermined binary masks
merge_thr = .8      # merging threshold, max correlation allowed
rf = 60             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 2           # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1          # downsampling factor in space for initialization,
low_rank_background = None  # None leaves background of each patch intact, WAS NONE
# #                     True performs global low-rank approximation if gnb>0
# gnb = 1             # number of background components (rank) if positive,
gnb = 0             # number of background components (rank) if positive, WAS 0
# #                     else exact ring model with following settings
# #                         gnb= 0: Return background as b and W
# #                         gnb=-1: Return full rank background B
# #                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
## For Hipp15
min_corr = .85       # min peak value from correlation image default .8
min_pnr = 8        # Hipp31 min peak to noise ration from PNR image, default 10
min_SNR = 2                                  # minimum SNR for accepting new components
rval_thr = 0.8                                # correlation threshold for new component inclusion
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor ## WAS 1.4
remove_very_bad_comps = True
del_duplicates = True

####################
normalize_init = True #  each pixels can be normalized by its median intensity
update_background_components=True
optimize_g = True # bool, default:False, flag for optimizing time constants
simultaneously = True # bool, optional, If true, demix and denoise/deconvolve simultaneously. Slower but can be more accurate
method_deconvolution = 'oasis' # 'oasis'|'cvxpy'|'cvx', default: 'oasis'
s_min = 25 # was None
#         s_min : float, optional, only applies to method 'oasis'
#             Minimal non-zero activity within each bin (minimal 'spike size').
#             For negative values the threshold is abs(s_min) * sn * sqrt(1-g)
#             If None (default) the standard L1 penalty is used
#             If 0 the threshold is determined automatically such that RSS <= sn^2 T

### Motion Correction
The background signal in micro-endoscopic data is very strong and makes the motion correction challenging. 
As a first step the algorithm performs a high pass spatial filtering with a Gaussian kernel to remove the bulk of the background and enhance spatial landmarks. 
The size of the kernel is given from the parameter `gSig_filt`. If this is left to the default value of `None` then no spatial filtering is performed (default option, used in 2p data).
After spatial filtering, the NoRMCorre algorithm is used to determine the motion in each frame. The inferred motion is then applied to the *original* data so no information is lost.

The motion corrected files are saved in memory mapped format. If no motion correction is being performed, then the file gets directly memory mapped.

In [6]:
## RUNNING CAIMAN ON INDIVIDUAL FILE
REMOVE_OLD_MEMMAP = True
REDO_MC = False

REDO_CNMFE = True

images = None # removing memory references in case this cell is rerun
Yr = None # removing memory references in case this cell is rerun
prev_mc_memmap  = None
prev_cmn_memmap = None
smin_vals = np.arange(-50, 50+1, 5)
# ddir = 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_18/16_44_33/MiniLFOV/'
# ddir = 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_10/18_25_10/MiniLFOV/'
for ddir in allMiniFolders:
    %reset_selective -f minFrame meanFrame maxFrame images mc cnm cn__filter pnr fullA Yr
    fnames         = ddir + '/msCam.tiff'#'\sub_MC.tiff'
    movie_tiffname = ddir + '/msCam_MC.tiff'
    saveName       = ddir + '/caiman_cnmfe_out.mat'
    sminName       = ddir + '/deconv_sweep.mat'
    cnm_fname      = ddir + '/cnm_data.hdf5'
    memmap_name    = glob.glob(ddir + '/memmap__*')
    mc_memmap_name    = glob.glob(ddir + '/msCam._els__*.mmap')
    
    # remove existing memmap files before running
    if memmap_name!= [] and REMOVE_OLD_MEMMAP == True: 
        memmap_name    = memmap_name[0].replace('\\', '/')
        os.remove(memmap_name)
    # remove existing memmap files before running
    if mc_memmap_name!= [] and REMOVE_OLD_MEMMAP==True:
        mc_memmap_name    = mc_memmap_name[0].replace('\\', '/')
        os.remove(mc_memmap_name)
    
    if os.path.isfile(saveName) and REDO_CNMFE==False:
        print('SKIPPING, FILE FOUND:  ' + saveName)
    else:
        # run analysis
#         %reset_selective -f minFrame meanFrame maxFrame images mc cnm cn__filter pnr fullA Yr
        ## params for file
        mc_dict = {
            'fnames': fnames,
            'fr': frate,
            'decay_time': decay_time,
            'pw_rigid': pw_rigid,
            'max_shifts': max_shifts,
            'gSig_filt': gSig_filt,
            'strides': strides,
            'overlaps': overlaps,
            'max_deviation_rigid': max_deviation_rigid,
            'border_nan': border_nan
        }
        opts = params.CNMFParams(params_dict=mc_dict)

        print('PW Motion Correction:  ' + fnames)

        ## do motion correction pw_rigid
        mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
        # mc.motion_correct(save_movie=True)
        mc.motion_correct(save_movie=True)
        fname_mc = mc.fname_tot_els if pw_rigid else mc.fname_tot_rig
        if pw_rigid:
            bord_px = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),
                                         np.max(np.abs(mc.y_shifts_els)))).astype(np.int)
        else:
            bord_px = np.ceil(np.max(np.abs(mc.shifts_rig))).astype(np.int)

        # save motion corrected video
        Yr= cm.load(fname_mc)
        Yr[Yr>255] = 255
        Yr[Yr<0]   = 0
        imwrite(movie_tiffname, np.round(Yr).astype('uint8')) # couldnt I use np.floor()
        Yr = None

        bord_px = 0 if border_nan is 'copy' else bord_px
        
        fname_new = cm.save_memmap(fname_mc, base_name='memmap_', order='C', border_to_0=bord_px)
        if pw_rigid:
            xshifts = mc.x_shifts_els
            yshifts = mc.y_shifts_els
        else:
            xshifts = mc.shifts_rig
            yshifts = 0

        ## RUN CNMF
        params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                    'K': K,
                                    'gSig': gSig,
                                    'gSiz': gSiz,
                                    'merge_thr': merge_thr,
                                    'p': p,
                                    'tsub': tsub,
                                    'ssub': ssub,
                                    'rf': rf,
                                    'stride': stride_cnmf,
                                    'only_init': True,    # set it to True to run CNMF-E
                                    'nb': gnb,
                                    'nb_patch': nb_patch,
                                    'method_deconvolution': method_deconvolution,       # could use 'cvxpy' alternatively
                                    'low_rank_background': low_rank_background,
                                    'update_background_components': True,  # sometimes setting to False improve the results
                                    'min_corr': min_corr,
                                    'min_pnr': min_pnr,
                                    'optimize_g': optimize_g,
                                    'simultaneously': simultaneously,
                                    's_min': s_min,
                                    'min_SNR': min_SNR,
                                    'rval_thr': rval_thr,
                                    'normalize_init': False,               # just leave as is
                                    'center_psf': True,                    # leave as is for 1 photon
                                    'ssub_B': ssub_B,
                                    'ring_size_factor': ring_size_factor,
                                    'del_duplicates': del_duplicates,                # whether to remove duplicates from initialization
                                    'remove_very_bad_comps': remove_very_bad_comps,                # whether to remove bad initial comps, see cnmf.py line 517
                                    'check_nan': False, # costly for large files
                                    'border_pix': bord_px}# number of pixels to not consider in the borders)
        opts.change_params(params_dict)         

        print('CNMF-E:  ' + fnames)
        Yr, dims, T = cm.load_memmap(fname_new)
        images = Yr.T.reshape((T,) + dims, order='F')

        minFrame = np.min(images, axis=0)
        meanFrame = np.mean(images, axis=0)
        maxFrame = np.max(images, axis=0)
        cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=gSig[0], swap_dim=False)
        ## Perform cnmfe
        cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
        cnm.fit(images)
        ## component eval
        min_SNR_eval = 2            # adaptive way to set threshold on the transient size
        r_values_min_eval = 0.8    # threshold on space consistency (if you lower more components
        #                        will be accepted, potentially with worst quality)
        cnm.params.set('quality', {'min_SNR': min_SNR_eval,
                                   'rval_thr': r_values_min_eval,
                                   'use_cnn': False})
        cnm.estimates.evaluate_components(images, cnm.params, dview=dview)
        print(' ***** ' + fnames)
        print('Number of total components: ', len(cnm.estimates.C))
        print('Number of accepted components: ', len(cnm.estimates.idx_components))
        cnm.save(cnm_fname)
        
        # Do a sweep of different s_min values for deconvolution
        S_Dict = {}
        for s_min_sub in smin_vals:
            if s_min_sub==0:
                s_min_sub = None
            cnm2 = cnm.deconvolve(s_min=s_min_sub)
            cnm2.estimates.evaluate_components(images, cnm.params, dview=dview)
            stemp = cnm2.estimates.S>0
            varName = 'smin_' + str(s_min_sub)
            varName = varName.replace('-', 'neg')
            compName_good = varName + '_idx_components'
            compName_bad  = varName + '_idx_components_bad'
            s = np.sum(np.sum(stemp, axis=1), axis=0)
            print(varName  + '      \tspikes = ' + str(s))
            S_Dict[varName] = stemp
            S_Dict[compName_good] = cnm2.estimates.idx_components
            S_Dict[compName_bad]  = cnm2.estimates.idx_components_bad

        scipy.io.savemat(sminName, S_Dict)
        
        # Save output in .mat
        fullA = scipy.sparse.csr_matrix.todense(cnm.estimates.A)

        mod_dict = params_dict
        for k,v in mod_dict.items():
            if mod_dict[k] == None:
                mod_dict[k] = 'None' # savemat doesnt work with None, convert to string
        garret_doesnt_like_hdf5 = {'A':cnm.estimates.A,
                               'fullA':fullA,
                               'C':cnm.estimates.C,
                               'S':cnm.estimates.S,
                               'YrA':cnm.estimates.YrA,                               
                               'idx_components':cnm.estimates.idx_components,
                               'idx_components_bad':cnm.estimates.idx_components_bad,
                               'neurons_sn':cnm.estimates.neurons_sn,
                               'sn':cnm.estimates.sn,
                               'SNR_comp':cnm.estimates.SNR_comp,
                               'cnn_preds':cnm.estimates.cnn_preds,
                               'dims':cnm.estimates.dims,
                               'r_values':cnm.estimates.r_values,
                               'lam':cnm.estimates.lam,
                               'corr_im':cn_filter,
                               'pnr_im':pnr,
                               'bord_px':bord_px,
                               'minFrame':minFrame,
                               'meanFrame':meanFrame,
                               'maxFrame':maxFrame,
                               'cn_filter':cn_filter,
                               'pnr':pnr,
                               'source_tiff':fnames,
                               'motion_corr_tiff':movie_tiffname,
                               'mc_xshifts':xshifts,
                               'mc_yshifts':yshifts,
                               'params':mod_dict,
                               'filename':fnames}
        scipy.io.savemat(saveName, garret_doesnt_like_hdf5)
        
        # remove the mmemap files from the last loop iter while out of memory
        if prev_mc_memmap!=None and REMOVE_OLD_MEMMAP:
            os.remove(prev_mc_memmap)
        if prev_cmn_memmap!=None and REMOVE_OLD_MEMMAP:
            os.remove(prev_cmn_memmap)
        prev_mc_memmap   = fname_mc[0]
        prev_cmn_memmap  = fname_new
        

<>:73: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:73: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\gjb326\AppData\Local\Temp\ipykernel_9564\2101625616.py:73: SyntaxWarning: "is" with a literal. Did you mean "=="?
  bord_px = 0 if border_nan is 'copy' else bord_px
       67247 [params.py:                 set():973] [9564] Changing key fnames in group data from None to C:/Users/gjb326/Desktop/RecordingData/AlejandroGrau/TestMouse1/2022_05_07/17_09_41/MiniLFOV/msCam.tiff
       67248 [params.py:                 set():973] [9564] Changing key fr in group data from 30 to 11
       67248 [params.py:                 set():973] [9564] Changing key decay_time in group data from 0.4 to 2
       67248 [params.py:                 set():973] [9564] Changing key pw_rigid in group motion from False to True
       67249 [params.py:                 set():973] [9564] Changing key max_shifts in group motion from (6, 6) to (55, 55)
       67249 [params.py:                

PW Motion Correction:  C:/Users/gjb326/Desktop/RecordingData/AlejandroGrau/TestMouse1/2022_05_07/17_09_41/MiniLFOV/msCam.tiff


       67496 [movies.py:      extract_shifts():357] [9564] Movie average is negative. Removing 1st percentile.
       67524 [movies.py:        apply_shifts():453] [9564] cubic interpolation
       67619 [movies.py:      extract_shifts():356] [9564] min_val in extract_shifts: -0.2392245849967003
       67619 [movies.py:      extract_shifts():357] [9564] Movie average is negative. Removing 1st percentile.
       67662 [movies.py:        apply_shifts():453] [9564] cubic interpolation
       67770 [movies.py:      extract_shifts():356] [9564] min_val in extract_shifts: -0.2392245849967003
       67771 [movies.py:      extract_shifts():357] [9564] Movie average is negative. Removing 1st percentile.
       67812 [movies.py:        apply_shifts():453] [9564] cubic interpolation
       67846 [motion_correction.py:motion_correct_batch_rigid():2816] [9564] Adding to movie 0.83279794
       67847 [motion_correction.py:motion_correct_batch_rigid():2822] [9564] 0
       67847 [motion_correction.py:

       67870 [motion_correction.py:motion_correction_piecewise():3144] [9564] Processing: frames: [1363 1364 1365 1366 1367 1368 1369 1370 1371 1372 1373 1374 1375 1376
 1377 1378 1379 1380 1381 1382 1383 1384 1385 1386 1387 1388 1389 1390
 1391 1392 1393 1394 1395 1396 1397 1398 1399 1400 1401 1402 1403 1404
 1405 1406 1407 1408 1409 1410 1411 1412 1413 1414 1415 1416 1417 1418
 1419 1420 1421 1422 1423 1424 1425 1426 1427 1428 1429 1430 1431 1432
 1433 1434 1435 1436 1437 1438 1439 1440 1441 1442]
       67871 [motion_correction.py:motion_correction_piecewise():3144] [9564] Processing: frames: [1443 1444 1445 1446 1447 1448 1449 1450 1451 1452 1453 1454 1455 1456
 1457 1458 1459 1460 1461 1462 1463 1464 1465 1466 1467 1468 1469 1470
 1471 1472 1473 1474 1475 1476 1477 1478 1479 1480 1481 1482 1483 1484
 1485 1486 1487 1488 1489 1490 1491 1492 1493 1494 1495 1496 1497 1498
 1499 1500 1501 1502 1503 1504 1505 1506 1507 1508 1509 1510 1511 1512
 1513 1514 1515 1516 1517 1518 1519 1520 1

       67876 [motion_correction.py:motion_correction_piecewise():3144] [9564] Processing: frames: [2723 2724 2725 2726 2727 2728 2729 2730 2731 2732 2733 2734 2735 2736
 2737 2738 2739 2740 2741 2742 2743 2744 2745 2746 2747 2748 2749 2750
 2751 2752 2753 2754 2755 2756 2757 2758 2759 2760 2761 2762 2763 2764
 2765 2766 2767 2768 2769 2770 2771 2772 2773 2774 2775 2776 2777 2778
 2779 2780 2781 2782 2783 2784 2785 2786 2787 2788 2789 2790 2791 2792
 2793 2794 2795 2796 2797 2798 2799 2800 2801 2802]
       67876 [motion_correction.py:motion_correction_piecewise():3144] [9564] Processing: frames: [2803 2804 2805 2806 2807 2808 2809 2810 2811 2812 2813 2814 2815 2816
 2817 2818 2819 2820 2821 2822 2823 2824 2825 2826 2827 2828 2829 2830
 2831 2832 2833 2834 2835 2836 2837 2838 2839 2840 2841 2842 2843 2844
 2845 2846 2847 2848 2849 2850 2851 2852 2853 2854 2855 2856 2857 2858
 2859 2860 2861 2862 2863 2864 2865 2866 2867 2868 2869 2870 2871 2872
 2873 2874 2875 2876 2877 2878 2879 2880 2

...

smin_40      	spikes = 2242


      274912 [components_evaluation.py:estimate_components_quality():800] [9564] Component evaluation in parallel


smin_45      	spikes = 1755


      276019 [components_evaluation.py:estimate_components_quality():800] [9564] Component evaluation in parallel


smin_50      	spikes = 1412


In [ ]:

# Save output in .mat
fullA = scipy.sparse.csr_matrix.todense(cnm.estimates.A)

mod_dict = params_dict
for k,v in mod_dict.items():
    if mod_dict[k] == None:
        mod_dict[k] = 'None' # savemat doesnt work with None, convert to string
garret_doesnt_like_hdf5 = {'A':cnm.estimates.A.astype('float64'),
                       'fullA':fullA,
                       'C':cnm.estimates.C,
                       'S':cnm.estimates.S,
                       'YrA':cnm.estimates.YrA,                               
                       'idx_components':cnm.estimates.idx_components,
                       'idx_components_bad':cnm.estimates.idx_components_bad,
                       'neurons_sn':cnm.estimates.neurons_sn,
                       'sn':cnm.estimates.sn,
                       'SNR_comp':cnm.estimates.SNR_comp,
                       'cnn_preds':cnm.estimates.cnn_preds,
                       'dims':cnm.estimates.dims,
                       'r_values':cnm.estimates.r_values,
                       'lam':cnm.estimates.lam,
                       'corr_im':cn_filter,
                       'pnr_im':pnr,
                       'bord_px':bord_px,
                       'minFrame':minFrame,
                       'meanFrame':meanFrame,
                       'maxFrame':maxFrame,
                       'cn_filter':cn_filter,
                       'pnr':pnr,
                       'source_tiff':fnames,
                       'motion_corr_tiff':movie_tiffname,
                       'mc_xshifts':xshifts,
                       'mc_yshifts':yshifts,
                       'params':mod_dict,
                       'filename':fnames}
scipy.io.savemat(saveName, garret_doesnt_like_hdf5)

# remove the mmemap files from the last loop iter while out of memory
if prev_mc_memmap!=None and REMOVE_OLD_MEMMAP:
    os.remove(prev_mc_memmap)
if prev_cmn_memmap!=None and REMOVE_OLD_MEMMAP:
    os.remove(prev_cmn_memmap)
prev_mc_memmap   = fname_mc[0]
prev_cmn_memmap  = fname_new

In [ ]:
images = None
Yr = None

In [ ]:
# import matplotlib.animation as animation

# # Yr= cm.load(fname_mc, outtype='uint8')
# Yr= cm.load(fname_mc)
# # Yt = Yr[7100:7199, 174:192, 260:275]
# fig = plt.figure()

    
# ims = []
# for im in Yt:
#     imt = plt.imshow(im, animated=True, vmin=0, vmax=255)
#     ims.append([imt])

# ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True,
#                                 repeat_delay=1000)

# plt.show()

In [ ]:
# if os.path.exists(movie_tiffname) and run_cnmfe and not resave_MC:
#     bord_px = 0
#     fname_mc = []
#     if bool(memmap_name): # if memmap file exists use it
#         fname_new = memmap_name[0]
#         print('Skipping MC, using existing memmap file: ' + fname_new)
#     else: # if not memmap MC file
#         print('Skipping MC, saving ' + movie_tiffname + ' to memmap...')
#         fname_new = cm.save_memmap(filenames=[movie_tiffname], dview=dview, base_name='memmap_', order='C', border_to_0=bord_px)
#     xshifts = 0
#     yshifts = 0
# else:
#     print('MC file exists and not running cnmf-e, skipping - ' +  movie_tiffname)
# if bool(fname_mc):
#     os.remove(fname_mc[0])

In [ ]:
# parameters for source extraction and deconvolution
p = 1#2?               # order of the autoregressive system ########## default 1
K = None            # upper bound on number of components per patch, in general None
gSig = (4,4)       # gaussian width of a 2D gaussian kernel, which approximates a neuron default=(3,3) # (3,3) for hipp6-9
                    # expected half size of neurons
    # (4,4) looks very good for Hipp16942
gSiz = (13, 13)     # average diameter of a neuron, in general 4*gSig+1 default=(13,13)
Ain = None          # possibility to seed with predetermined binary masks
merge_thr = .8      # merging threshold, max correlation allowed
rf = 60             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 2           # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1          # downsampling factor in space for initialization,
low_rank_background = None  # None leaves background of each patch intact, WAS NONE
# #                     True performs global low-rank approximation if gnb>0
# gnb = 1             # number of background components (rank) if positive,
gnb = 0             # number of background components (rank) if positive, WAS 0
# #                     else exact ring model with following settings
# #                         gnb= 0: Return background as b and W
# #                         gnb=-1: Return full rank background B
# #                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
## For Hipp15
min_corr = .85       # min peak value from correlation image default .8
min_pnr = 8        # Hipp31 min peak to noise ration from PNR image, default 10
min_SNR = 2                                  # minimum SNR for accepting new components
rval_thr = 0.8                                # correlation threshold for new component inclusion
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor ## WAS 1.4
remove_very_bad_comps = True
del_duplicates = True

####################
normalize_init = True #  each pixels can be normalized by its median intensity
update_background_components=True
optimize_g = True # bool, default:False, flag for optimizing time constants
simultaneously = True # bool, optional, If true, demix and denoise/deconvolve simultaneously. Slower but can be more accurate
method_deconvolution = 'oasis' # 'oasis'|'cvxpy'|'cvx', default: 'oasis'
s_min = 25 # was None
#         s_min : float, optional, only applies to method 'oasis'
#             Minimal non-zero activity within each bin (minimal 'spike size').
#             For negative values the threshold is abs(s_min) * sn * sqrt(1-g)
#             If None (default) the standard L1 penalty is used
#             If 0 the threshold is determined automatically such that RSS <= sn^2 T

In [ ]:
params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                            'K': K,
                            'gSig': gSig,
                            'gSiz': gSiz,
                            'merge_thr': merge_thr,
                            'p': p,
                            'tsub': tsub,
                            'ssub': ssub,
                            'rf': rf,
                            'stride': stride_cnmf,
                            'only_init': True,    # set it to True to run CNMF-E
                            'nb': gnb,
                            'nb_patch': nb_patch,
                            'method_deconvolution': method_deconvolution,       # could use 'cvxpy' alternatively
                            'low_rank_background': low_rank_background,
                            'update_background_components': True,  # sometimes setting to False improve the results
                            'min_corr': min_corr,
                            'min_pnr': min_pnr,
                            'optimize_g': optimize_g,
                            'simultaneously': simultaneously,
                            's_min': s_min,
                            'min_SNR': min_SNR,
                            'rval_thr': rval_thr,
                            'normalize_init': False,               # just leave as is
                            'center_psf': True,                    # leave as is for 1 photon
                            'ssub_B': ssub_B,
                            'ring_size_factor': ring_size_factor,
                            'del_duplicates': del_duplicates,                # whether to remove duplicates from initialization
                            'remove_very_bad_comps': remove_very_bad_comps,                # whether to remove bad initial comps, see cnmf.py line 517
                            'check_nan': False, # costly for large files
                            'border_pix': bord_px}# number of pixels to not consider in the borders)
opts.change_params(params_dict)         

print('CNMF-E:  ' + fnames)
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')

cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=gSig[0], swap_dim=False)
cm.utils.visualization.inspect_correlation_pnr(cn_filter, pnr);


In [ ]:
## RUN CNMF
params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                            'K': K,
                            'gSig': gSig,
                            'gSiz': gSiz,
                            'merge_thr': merge_thr,
                            'p': p,
                            'tsub': tsub,
                            'ssub': ssub,
                            'rf': rf,
                            'stride': stride_cnmf,
                            'only_init': True,    # set it to True to run CNMF-E
                            'nb': gnb,
                            'nb_patch': nb_patch,
                            'method_deconvolution': method_deconvolution,       # could use 'cvxpy' alternatively
                            'low_rank_background': low_rank_background,
                            'update_background_components': True,  # sometimes setting to False improve the results
                            'min_corr': min_corr,
                            'min_pnr': min_pnr,
                            'optimize_g': optimize_g,
                            'simultaneously': simultaneously,
                            's_min': s_min,
                            'min_SNR': min_SNR,
                            'rval_thr': rval_thr,
                            'normalize_init': False,               # just leave as is
                            'center_psf': True,                    # leave as is for 1 photon
                            'ssub_B': ssub_B,
                            'ring_size_factor': ring_size_factor,
                            'del_duplicates': del_duplicates,                # whether to remove duplicates from initialization
                            'remove_very_bad_comps': remove_very_bad_comps,                # whether to remove bad initial comps, see cnmf.py line 517
                            'check_nan': False, # costly for large files
                            'border_pix': bord_px}# number of pixels to not consider in the borders)
opts.change_params(params_dict)         

print('CNMF-E:  ' + fnames)
# Yr, dims, T = cm.load_memmap(fname_new)
# images = Yr.T.reshape((T,) + dims, order='F')
     
# minFrame = np.min(images, axis=0)
# meanFrame = np.mean(images, axis=0)
# maxFrame = np.max(images, axis=0)
# cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=gSig[0], swap_dim=False)
## Perform cnmfe
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
cnm.fit(images)
## component eval
min_SNR_eval = 2            # adaptive way to set threshold on the transient size
r_values_min_eval = 0.8    # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'min_SNR': min_SNR_eval,
                           'rval_thr': r_values_min_eval,
                           'use_cnn': False})
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)
print(' ***** ' + fnames)
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))
cnm.save(cnm_fname)

    ## Save output in .mat
# fullA = scipy.sparse.csr_matrix.todense(cnm.estimates.A)

# mod_dict = params_dict
# for k,v in mod_dict.items():
#     if mod_dict[k] == None:
#         mod_dict[k] = 'None' # savemat doesnt work with None, convert to string
# garret_doesnt_like_hdf5 = {'A':cnm.estimates.A,
#                        'fullA':fullA,
#                        'C':cnm.estimates.C,
#                        'S':cnm.estimates.S,
#                        'YrA':cnm.estimates.YrA,                               
#                        'idx_components':cnm.estimates.idx_components,
#                        'idx_components_bad':cnm.estimates.idx_components_bad,
#                        'neurons_sn':cnm.estimates.neurons_sn,
#                        'sn':cnm.estimates.sn,
#                        'SNR_comp':cnm.estimates.SNR_comp,
#                        'cnn_preds':cnm.estimates.cnn_preds,
#                        'dims':cnm.estimates.dims,
#                        'r_values':cnm.estimates.r_values,
#                        'lam':cnm.estimates.lam,
#                        'corr_im':cn_filter,
#                        'pnr_im':pnr,
#                        'bord_px':bord_px,
#                        'minFrame':minFrame,
#                        'meanFrame':meanFrame,
#                        'maxFrame':maxFrame,
#                        'source_tiff':fnames,
#                        'motion_corr_tiff':movie_tiffname,
#                        'mc_xshifts':xshifts,
#                        'mc_yshifts':yshifts,
#                        'params':mod_dict,
#                        'filename':fnames}
# scipy.io.savemat(saveName, garret_doesnt_like_hdf5)

In [ ]:
sessionLoop=5
fnames = ms_runfolder[sessionLoop] +          '\msCam.tiff'
movie_tiffname = ms_runfolder[sessionLoop] +  '\msCam_MC.tiff'
saveName = ms_runfolder[sessionLoop] +        '\caiman_cnmfe_out.mat'
cnm_fname = ms_runfolder[sessionLoop] +       '\cnm_data.hdf5'


# cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
# from caiman.source_extraction import cnmf

cmn = cnmf.cnmf.load_CNMF(filename='I:\\MiniData\\Hipp36\\cnm_data.hdf5', n_processes=n_processes, dview=dview)

In [ ]:
## component eval
min_SNR_eval = 2            # adaptive way to set threshold on the transient size
r_values_min_eval = 0.8    # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'min_SNR': min_SNR_eval,
                           'rval_thr': r_values_min_eval,
                           'use_cnn': False})
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)
print(' ***** ' + fnames)
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))
cnm.save(cnm_fname)

In [ ]:
cnm.estimates.plot_contours(img=pnr, idx=cnm.estimates.idx_components, thr=.6, thr_method='max');

In [ ]:
cnm.estimates.view_components(idx=cnm.estimates.idx_components);

In [ ]:
plt.figure()
cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=gSig[0], swap_dim=False)
plt.imshow(pnr)


In [ ]:
cm.utils.visualization.inspect_correlation_pnr(cn_filter, pnr)

In [ ]:
# fname_new = 'E:/Miniscope Data\mouse_cw2/2021_04_09/09_04_40/Miniscope/memmap__d1_972_d2_1296_d3_1_order_C_frames_2983_.mmap'
# Yr, dims, T = cm.load_memmap(fname_new)
# images = Yr.T.reshape((T,) + dims, order='F')
# movname = ms_runfolder[sessionLoop] + '/test.tiff'
cnm.estimates.play_movie(images, q_max=1000, q_min=0, gain_res=1, magnification=.5,
      bpx=20, use_color=False, # color makes it take forever
      include_bck=True);

In [ ]:
# for loading old caiman data
# sessionLoop=4
# fnames = all_filenames[sessionLoop]
# movie_tiffname = mc_save_name[sessionLoop]
# saveName = save_filenames[sessionLoop]
# cnm_fname = cnm_filenames[sessionLoop]
# from caiman.source_extraction.cnmf import cnmf 
# cnm = cnmf.load_CNMF(cnm_fname)

fname_new = 'E:/Miniscope Data\mouse_cw2/2021_04_09/09_04_40/Miniscope/memmap__d1_972_d2_1296_d3_1_order_C_frames_2983_.mmap'
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')

In [ ]:
params_dict={'s_min': -5,
    'min_SNR': min_SNR}
opts.change_params(params_dict)  

cnm.deconvolve()